In [1]:
import pandas as pd
from tqdm import tqdm
from glob import glob
import os
import pickle
os.chdir('../../encoder4editing/')

from scripts.inference import *

In [2]:
# Setup Model
model_path = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Models/e4e/experiments_default_lr/checkpoints/best_model.pt"
device = 'cuda'

net, opts = setup_model(model_path, device)
generator = net.decoder
generator.eval()

# Get transforms
dataset_args = data_configs.DATASETS[opts.dataset_type]
transforms_dict = dataset_args['transforms'](opts).get_transforms()

# Define inversion function
def encode_from_path(image_path):
    # Input
    img_orig = Image.open(image_path).convert('RGB')
    img = transforms_dict['transform_test'](img_orig)
    img = img.reshape(-1, 3, 256, 256)
    img = img.to('cuda').float()

    latent = get_latents(net, img)
    return latent

Loading e4e over the pSp framework from checkpoint: /pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Models/e4e/experiments_default_lr/checkpoints/best_model.pt


In [3]:
e4e_images_paths = glob("/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Zalando_Germany_Dataset/dresses/images/e4e_images/*/*.jpg")
print(f"Found {e4e_images_paths} images")


all_latents = torch.zeros(len(e4e_images_paths), 1, 16, 512)
all_paths = []
for i, path in enumerate(tqdm(e4e_images_paths)):
    sku = path.split('/')[-1].split('_')[-1].split('.')[0]
    latent = encode_from_path(path)
    latent = latent.cpu().detach()

    all_latents[i] = latent
    all_paths.append(path)

Found ['/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Zalando_Germany_Dataset/dresses/images/e4e_images/train/CE321C0DU-K11.jpg', '/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Zalando_Germany_Dataset/dresses/images/e4e_images/train/FAB21C095-K11.jpg', '/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Zalando_Germany_Dataset/dresses/images/e4e_images/train/IV321C0E5-Q11.jpg', '/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Zalando_Germany_Dataset/dresses/images/e4e_images/train/NN121C0H6-A11.jpg', '/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Zalando_Germany_Dataset/dresses/images/e4e_images/train/V1021C48D-H11.jpg', '/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Zalando_Germany_Dataset/dresses/images/e4e_images/train/GAG21C039-Q11.jpg', '/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Zalando_Germany_Dataset/dresses/images/e4e_images/train/SHI21C015-G11.jpg', '/pfs/work7/workspace/scratch/tu_zxmav84-

100%|██████████| 14060/14060 [17:48<00:00, 13.16it/s]


In [4]:
save_path = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Models/e4e/experiments_default_lr/inversions/"
# Save latents tensor
torch.save(all_latents, f"{save_path}latents.pt")
# save file paths
with open(f"{save_path}file_paths.pkl", 'wb') as handle:
    pickle.dump(all_paths, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Create Dictionary with latents

In [ ]:
latents = torch.load(f"{save_path}latents.pt")
with open(f"{save_path}file_paths.pkl", 'rb') as f:
    file_paths = pickle.load(f)

latents_dict = {}
for i, file in enumerate(file_paths):
    sku = file.split('/')[-1].split('_')[-1].split('.')[0]
    latents_dict[sku] = latents[i]